In [ ]:
# Import libraries:
import pandas as pd
import importlib
import dt_ids7_export_module as bh_utils
import mapping_module as bh_map
import plot_module as bh_plot
import reporting_module as bh_report
import mapping_dict_rad_xa as rad_xa_dict

# Make sure personal libraries are reloaded incase of adjustments:
importlib.reload(bh_utils)
importlib.reload(bh_map)
importlib.reload(bh_plot)
importlib.reload(bh_report)
importlib.reload(rad_xa_dict)

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path, hostname

# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)
# Merge the dataframes:
data = bh_utils.merge_ids7_dt(df_ids7, df_dt, True)
# Get the mapping dictionary:
mapping_dict = rad_xa_dict.get_rad_xa_mapping_dict()
# First we perform mapping on selected procedures:
data = bh_map.map_procedures(data, mapping_dict, True)

del df_dt, df_ids7, mapping_dict

In [ ]:
unmapped = data[data['Mapped Procedures'] == 'Unmapped']

In [ ]:
bh_utils.export_examination_codes_to_text_file(data, laboratory=None)

In [ ]:
mapping_dict = rad_xa_dict.get_rad_xa_mapping_dict()
export = False
# Make a list of all the unique values in the mapping dictionary 
# (make list, convert to set, and make list again to remove duplicates):
procedures = list(set(list(mapping_dict.values())))
# Sort the list:
procedures.sort()
for p in procedures:
    bh_plot.plot_representative_dose(data, p, 0, export)

In [ ]:
# Remove unmapped and cochlia examinations from the dataframe
elements_to_remove = ['Unmapped', 'Cochlia', 'Scoliose', 'TAVI']

mask = ~data['Mapped Procedures'].isin(elements_to_remove)
bh_plot.plot_representative_dose_by_procedure(data[mask], 500, export)

In [ ]:
# Remove unmapped and cochlia examinations from the dataframe

data_rf2 = data[data['Modality Room'] == 'RRH_RF2']
elements_to_remove = ['Unmapped', 'Nefrostomi innleggelse', 'RG Abdomen - Abscessografi, Fjerning/Skifte av dren', 'RG Columna eks. Myelografi']

mask = ~data_rf2['Mapped Procedures'].isin(elements_to_remove)
bh_plot.plot_representative_dose_by_procedure(data_rf2[mask], 20, export)

In [ ]:
data_abdomen = data[(data['Modality Room'] == 'RRH_XA2') | (data['Modality Room'] == 'RRH_XA4') | (data['Modality Room'] == 'IRH_XA7')]
elements_to_remove = ['Unmapped', 'Caput Embolisering', 'Caput og collum', 'Myelografi', 'RG Columna eks. Myelografi', 'Trombektomi',
                      'Cochlia', 'Scoliose', 'TAVI']

mask = ~data_abdomen['Mapped Procedures'].isin(elements_to_remove)
bh_plot.plot_representative_dose_by_procedure(data_abdomen[mask], 500, export)

In [ ]:
data_nevro = data[(data['Modality Room'] == 'RRH_XA2') | (data['Modality Room'] == 'RRH_XA4')]
elements_to_remove = ['Unmapped', 'Abdomen/Bekken/Aorta PTA/Embolisering/Stent/Stentgraft',
                      'EVAR', 'TEVAR', 'BEVAR/FEVAR', 'Lever', 'Milt/Nyrer', 'Nefrostomi innleggelse', 'Nefrostomi/Pyelografi skifte eller fjerning',
                      'Overex', 'PTC/PTBD', 'Pulmonalarterier/PTA/Embolisering', 'RG Abdomen - Abscessografi, Fjerning/Skifte av dren',
                      'TACE', 'Underex', 'Urethragrafi/Urografi/Urinveier/MUCG']

mask = ~data_nevro['Mapped Procedures'].isin(elements_to_remove)
bh_plot.plot_representative_dose_by_procedure(data_nevro[mask], 500, export)

In [ ]:
procedure = 'EVAR'
lab = 'RRH_XA4'
# Get median f+A time for the procedure:
median_fa_time = data[(data['Mapped Procedures'] == procedure) & (data['Modality Room'] == lab)]['F+A Time (s)'].median()
print('Median f+A time for ' + procedure + 'on lab ' + lab + ' is ' + str(median_fa_time/60) + ' minutes.')